# Notebook 04: Bias Mitigation

Apply reweighing from AIF360 and compare fairness metrics.

In [ ]:
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Prepare AIF360-compatible dataset
aif_df = df.copy()
aif_df['credit_risk'] = aif_df['credit_risk'].map({1: 1, 2: 0})

dataset_orig = StandardDataset(
    aif_df,
    label_name='credit_risk',
    favorable_classes=[1],
    protected_attribute_names=['foreign_worker'],
    privileged_classes=[[1]]
)

# Apply Reweighing
RW = Reweighing(
    privileged_groups=[{'foreign_worker': 1}],
    unprivileged_groups=[{'foreign_worker': 0}]
)
dataset_transf = RW.fit_transform(dataset_orig)

# Train model on reweighted data
X_rw = dataset_transf.features
y_rw = dataset_transf.labels.ravel()
sample_weights = dataset_transf.instance_weights

model_rw = LogisticRegression(max_iter=1000)
model_rw.fit(X_rw, y_rw, sample_weight=sample_weights)
y_pred_rw = model_rw.predict(X_rw)

print('Accuracy after reweighing:', accuracy_score(y_rw, y_pred_rw))